In [ ]:
!pip install catboost
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
import gc
from sklearn.model_selection import train_test_split

In [ ]:
# 이상치 처리 X
train_df=pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/제주/mid_train_df(2).csv')
#test_df=pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/제주/mid_test_df(2).csv')

sub = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/제주/submission.csv')

In [ ]:
train_df.fillna(9999,inplace=True)
#test_df.fillna(9999,inplace=True)

In [ ]:
train_df.drop(['CSTMR_CNT','CNT'],axis=1,inplace=True)
train_df.head()

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,AMT,STD_total,SIDO_STD_total,SIDO_CCG_STD_total,ASF_total,SIDO_ASF_total,SIDO_CCG_ASF_total,SIDO_ASF_2_total,SIDO_CCG_ASF_2_total,STD_2019,SIDO_STD_2019,SIDO_CCG_STD_2019,ASF_2019,SIDO_ASF_2019,SIDO_CCG_ASF_2019,SIDO_ASF_2_2019,SIDO_CCG_ASF_2_2019,STD_2020,SIDO_STD_2020,SIDO_CCG_STD_2020,ASF_2020,SIDO_ASF_2020,SIDO_CCG_ASF_2020,SIDO_ASF_2_2020,SIDO_CCG_ASF_2_2020
0,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,311200,16,258,2121,11,14,357,201,1734,10,33,2140,11,196,338.0,225.0,2027.0,13,244.0,3128.0,6,49.0,36.0,117.0,1528.0
1,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,1374500,16,258,2121,0,24,0,50,173,10,33,2140,2,53,15.0,4.0,812.0,13,244.0,3128.0,3,42.0,19.0,79.0,780.0
2,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,818700,16,258,2121,2,32,10,179,173,10,33,2140,0,5,15.0,4.0,812.0,13,244.0,3128.0,0,219.0,43.0,127.0,104.0
3,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,1717000,16,258,2121,0,24,0,41,282,10,33,2140,2,1,15.0,99.0,58.0,13,244.0,3128.0,3,42.0,742.0,114.0,3174.0
4,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,1047300,16,258,2121,0,51,0,226,14,10,33,2140,2,10,314.0,15.0,5.0,13,244.0,3128.0,3,51.0,19.0,1.0,209.0


In [ ]:
# test_df 정렬
#test_df = test_df[list(train_df.drop('AMT',axis=1).columns)]
#test_df.head()

In [ ]:
def make_encoders(df):
  encoders={}
  for col in df.drop(['AMT'],axis=1).columns:
    con_list =[]
    encoder={}
    for idx, content in enumerate(df[col].unique()):
      encoder[content]=idx

    if col == 'REG_YYMM':
        encoder[202005] = idx+1
        encoder[202006] = idx+2
        encoder[202007] = idx+3
     
    encoders[col] = encoder
  
  return encoders

In [ ]:
# previous 3,6 변수를 만들어서 적용할 것이기 때문에 19년 7월 아래로는 제거해주자
train_df = train_df[train_df['REG_YYMM']>201906]

In [ ]:
# public은 4월 밖에 계산을 못함

df_not04=train_df[train_df['REG_YYMM'] != 202004]
df_202004=train_df[train_df['REG_YYMM'] == 202004]

# 5월,6월 7월 출력하기
df_form= df_202004.drop('REG_YYMM',axis=1).copy()
df_form['REG_YYMM']=202005
df_202005=df_form.copy()
df_form['REG_YYMM']=202006
df_202006=df_form.copy()
df_form['REG_YYMM']=202007
df_202007=df_form.copy()

# 정렬

for df in [df_202005,df_202006,df_202007]:
  df = df[list(df_202004.columns)]

del df_form,df_202004
gc.collect()

22

In [ ]:
# 인코딩
encoders = make_encoders(train_df)

In [ ]:
for col in encoders.keys():
  #df_not04[col] = df_not04[col].map(encoders[col])
  #df_202004[col] = df_202004[col].map(encoders[col])
  train_df[col] = train_df[col].map(encoders[col])
  #df_202005[col] = df_202005[col].map(encoders[col])
  #df_202006[col] = df_202006[col].map(encoders[col])
  #df_202007[col] = df_202007[col].map(encoders[col])

In [ ]:
for col in train_df.columns:
  if col !='AMT':
    #df_not04[col] = df_not04[col].astype('object')
    #df_202004[col] = df_202004[col].astype('object')
    train_df[col] = train_df[col].astype('object')
    #df_202005[col] = df_202005[col].astype('object')
    #df_202006[col] = df_202006[col].astype('object')
    #df_202007[col] = df_202007[col].astype('object')

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(train_df.drop('AMT',axis=1),
                                                  np.log1p(train_df['AMT']), test_size=0.3, random_state=42)

In [ ]:
del train_df
gc.collect()

257

catboost 파라미터: bagging_temperature ,
              depth , 
                  learning_rate ,
                  min_data_in_leaf , 
                  max_leaves , 
                  l2_leaf_reg , 
                  border_count

In [ ]:
cat_features = list(X_train.columns)

model = CatBoostRegressor(loss_function="RMSE",
                           eval_metric="RMSE",
                           task_type="GPU",
                           learning_rate=0.01,
                           iterations=200,
                           boosting_type='Plain', # 메모리 과부하 방지
                           #max_ctr_complexity=2,
                           gpu_cat_features_storage = 'CpuPinnedMemory',
                           early_stopping_rounds=10)



model.fit(X_train, y_train,
          cat_features=cat_features,
          eval_set=(X_val, y_val),
          use_best_model=True,
          verbose = 10)

0:	learn: 1.7132157	test: 1.7118699	best: 1.7118699 (0)	total: 99.4ms	remaining: 9.84s
10:	learn: 1.6951295	test: 1.6937251	best: 1.6937251 (10)	total: 947ms	remaining: 7.66s
20:	learn: 1.6799799	test: 1.6785782	best: 1.6785782 (20)	total: 1.79s	remaining: 6.75s
30:	learn: 1.6635969	test: 1.6621243	best: 1.6621243 (30)	total: 2.62s	remaining: 5.84s
40:	learn: 1.6475225	test: 1.6459555	best: 1.6459555 (40)	total: 3.46s	remaining: 4.97s
50:	learn: 1.6337382	test: 1.6320971	best: 1.6320971 (50)	total: 4.29s	remaining: 4.12s
60:	learn: 1.6216671	test: 1.6199613	best: 1.6199613 (60)	total: 5.12s	remaining: 3.27s
70:	learn: 1.6112898	test: 1.6095052	best: 1.6095052 (70)	total: 5.95s	remaining: 2.43s
80:	learn: 1.6000801	test: 1.5982725	best: 1.5982725 (80)	total: 6.82s	remaining: 1.6s
90:	learn: 1.5909631	test: 1.5891127	best: 1.5891127 (90)	total: 7.66s	remaining: 758ms
99:	learn: 1.5840125	test: 1.5821479	best: 1.5821479 (99)	total: 8.43s	remaining: 0us
bestTest = 1.58214788
bestIteration 

In [ ]:
del X_train,y_train,X_val,y_val
gc.collect()

0

In [ ]:
model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,STD_CLSS_NM,28.007523
1,SIDO_CCG_STD_2020,25.086554
2,HOM_SIDO_NM,5.679408
3,CARD_SIDO_NM,5.665522
4,HOM_CCG_NM,5.099663
5,SIDO_CCG_ASF_2_total,4.306144
6,SIDO_CCG_ASF_2_2020,3.747939
7,CARD_CCG_NM,3.504450
8,SIDO_CCG_STD_total,3.378088
9,SIDO_CCG_ASF_2_2019,2.770188


# submit

In [ ]:
# for df in [df_202005,df_202006,df_202007]:
#   pred=model.predict(df.drop('AMT',axis=1))
#   df['AMT_pred'] = pred
#   df['AMT_pred'] = np.expm1(df['AMT_pred'])

In [ ]:
# # decoder 만들기

# decoders={}
# for kk,vv in encoders.items():
#   decoder={}
#   for k,v in vv.items():
#     decoder[v]=k
#   decoders[kk] = decoder

In [ ]:
# for col in decoders.keys():
#   #df_202004[col] = df_202004[col].map(decoders[col])
#   df_202005[col] = df_202005[col].map(decoders[col])
#   df_202006[col] = df_202006[col].map(decoders[col])
#   df_202007[col] = df_202007[col].map(decoders[col])

In [ ]:
# sub_form=sub.drop(['id','AMT'],axis=1).copy()
# sub_form['REG_YYMM']=202005
# df_05=df_202005.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'])['AMT_pred'].sum().reset_index(name='AMT')
# sub_05 = sub_form.merge(df_05,on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'],how='left')
# sub_05.fillna(0,inplace=True)

# sub_form=sub.drop(['id','AMT'],axis=1).copy()
# sub_form['REG_YYMM']=202006
# df_06=df_202006.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'])['AMT_pred'].sum().reset_index(name='AMT')
# sub_06 = sub_form.merge(df_06,on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'],how='left')
# sub_06.fillna(0,inplace=True)

# sub_form=sub.drop(['id','AMT'],axis=1).copy()
# sub_form['REG_YYMM']=202007
# df_07=df_202007.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'])['AMT_pred'].sum().reset_index(name='AMT')
# sub_07 = sub_form.merge(df_07,on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'],how='left')
# sub_07.fillna(0,inplace=True)

In [ ]:
# sub_05.to_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/제주/submission_baseline_cat(200,5월).csv', encoding='utf-8-sig',index=False)
# sub_06.to_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/제주/submission_baseline_cat(200,6월).csv', encoding='utf-8-sig',index=False)
# sub_07.to_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/제주/submission_baseline_cat(200,7월).csv', encoding='utf-8-sig',index=False)

---

단독계산

In [ ]:
# df_t = df_202004.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'])['AMT_pred'].sum().reset_index(name='AMT')
# df_t['AMT'] = df_t['AMT'].fillna(0)
# df_t['AMT'] = df_t['AMT'].astype('int')

In [ ]:
# sub.drop('AMT',axis=1,inplace=True)
# sub = sub.merge(df_t,on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'],how='left')
# sub.fillna(0,inplace=True)

In [ ]:
# sub.head()

In [ ]:
# sub.to_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/제주/submission_baseline_cat(300).csv', encoding='utf-8-sig',index=False)

In [ ]:
# pred=model.predict(df_202004.drop('AMT',axis=1))
# df_202004['AMT_pred'] = pred
# df_202004['AMT_pred'] = np.expm1(df_202004['AMT_pred'])